# Import dependencies

## Set up Spark

In [1]:
kernel.silent(true)

In [2]:
import $ivy.`org.apache.spark::spark-sql:3.1.2`
import $ivy.`org.apache.spark::spark-mllib:3.1.2`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark.sql._
import org.apache.spark.ml._
import org.apache.spark.sql.functions._

val spark = {
  SparkSession.builder()
    .master("local[*]")
    .getOrCreate()
}

## Add depencies

In [3]:
import coursierapi._
interp.repositories() ++= Seq(
  MavenRepository.of("https://dl.cloudsmith.io/public/gisaia/public/maven/"),
  MavenRepository.of("https://repo.osgeo.org/repository/release/"),
  MavenRepository.of("https://repository.jboss.org/maven2/")
)

## Import Geotools

In [4]:
import $ivy.`org.geotools:gt-referencing:20.1`
import $ivy.`org.geotools:gt-geometry:20.1`
import $ivy.`org.geotools:gt-epsg-hsql:20.1`

### Declare display functions

In [5]:
import $ivy.`org.plotly-scala::plotly-almond:0.5.4`
import plotly._, plotly.element._, plotly.layout._, plotly.Almond._


implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit:Int = 20, truncate: Int = 20) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map { cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }

        publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map { row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

## Import arlas-proc

In [6]:
import $ivy.`io.arlas:arlas-proc:0.6.1-SNAPSHOT`

# Apply processing pipeline on vessel location sample (AIS data)

## Import ARLAS-proc library functions

In [7]:
import io.arlas.data.sql._
import io.arlas.data.sql.ArlasDataFrame
import io.arlas.data.model._
import io.arlas.data.transform._
import io.arlas.data.transform.fragments._
import io.arlas.data.transform.features._
import io.arlas.data.transform.timeseries._
import io.arlas.data.transform.ml._
import io.arlas.data.transform.tools._

# Run processing pipeline on vessel location sample

The Automatic Identification System (AIS) is a safety system that records and broadcasts vessels location.

These messages are received by other vessels and it helps them to avoid collision, particularly when the visibility is low. It also contains information about the vessel dynamics and characteristics.

<p align="left">
  <img src="./images/ais_system.png" width=700" title="AIS System" style="float:center">
</p>

The Danish Maritime Authority [provide AIS messages](https://www.dma.dk/SikkerhedTilSoes/Sejladsinformation/AIS/Sider/default.aspx) received around danish coast that we use to illustrate this maritime data analysis, in accordance with the conditions for the use of Danish public data.

Here we select two vessels ...

- ### Extract raw data from csv

In [8]:
val raw_data = readFromCsv(spark, delimiter=",", sources="/home/jovyan/work/data/extract_2_ids.csv")
        .select("# Timestamp", "MMSI", "Latitude", "Longitude", "SOG", "COG", "Heading", "IMO", "Callsign", "Name",
                "Ship type", "Cargo type", "Width", "Length")
raw_data.showHTML(10);

# Timestamp,MMSI,Latitude,Longitude,SOG,COG,Heading,IMO,Callsign,Name,Ship type,Cargo type,Width,Length
02/01/2021 00:00:01,220343000,57.717728,10.585658,0.0,345.7,232,Unknown,null,null,Undefined,null,null,null
02/01/2021 00:00:05,220343000,57.717728,10.585658,0.0,345.7,232,Unknown,OU4649,SOFIE LYKKE,Fishing,null,4,16
02/01/2021 00:00:09,219948000,57.720573,10.594807,0.0,355.0,160,Unknown,null,null,Undefined,null,null,null
02/01/2021 00:00:11,220343000,57.717723,10.585655,0.0,286.7,232,Unknown,OU4649,SOFIE LYKKE,Fishing,null,4,16
02/01/2021 00:00:20,219948000,57.720577,10.594807,0.0,348.2,160,Unknown,null,null,Undefined,null,null,null
02/01/2021 00:00:21,220343000,57.717720,10.585642,0.0,9.2,232,Unknown,OU4649,SOFIE LYKKE,Fishing,null,4,16
02/01/2021 00:00:29,219948000,57.720577,10.594807,0.0,6.5,160,Unknown,null,null,Undefined,null,null,null
02/01/2021 00:00:31,220343000,57.717715,10.585647,0.0,243.9,232,Unknown,OU4649,SOFIE LYKKE,Fishing,null,4,16
02/01/2021 00:00:39,219948000,57.720573,10.594808,0.0,20.2,160,Unknown,null,null,Undefined,null,null,null
02/01/2021 00:00:40,220343000,57.717715,10.585648,0.0,312.8,232,Unknown,OU4649,SOFIE LYKKE,Fishing,null,4,16


We observe basic info contained in AIS messages:
- `# Timestamp`: Time of the emmited ais message
- `Latitude` and `Longitude`: Location of the vessel when emmiting AIS message
- `SOG`: Speed over ground (knot) of the vessel
- `COG`: Course over ground (degree from north), the direction of the vessel move
- `Heading`: The direction(degree from north) of the vessel head (can be different of the COG if the vessel is drafting)
- `MMSI`: Unique identifier (Maritime Mobile Service Identity) of the vessel
- `IMO`: Other unique identifier (International Maritime Organization) of the vessel
- `Callsign`: Other unique identifier of the vessel
- `Name`: Readable name of the vessel (not unique)
- `Ship type`: Ship type of the vessel (fishing, passenger, cargo...)
- `Cargo type`: Additional vessel type info for cargos
- `Width` and `Length`: Dimension of the vessel

Some of these info are not alway filled.

We can plot these vessel raw locations on Qgis:

<p align="left">
  <img src="./images/qgis_raw.png" width="900" title="Raw location" style="float:left">
</p>


Here we see the two vessels locations in the port of Skagen (DK).

- ### Format data by setting numeric variable types and processing unix timestamp


Set the object identifier, latitude, longitude and date column to handle moving object data properly

In [9]:
val dataModel = DataModel(
  idColumn = "MMSI",
  latColumn = "Latitude",
  lonColumn = "Longitude",
  timestampColumn = "# Timestamp",
  timeFormat = "dd/MM/yyyy HH:mm:ss"
);

val formatted_data = raw_data.asArlasFormattedData(dataModel, doubleColumns = Vector("SOG", "COG", "Heading"))

formatted_data.sort("MMSI", "arlas_timestamp").showHTML(10)

# Timestamp,MMSI,Latitude,Longitude,SOG,COG,Heading,IMO,Callsign,Name,Ship type,Cargo type,Width,Length,arlas_timestamp,arlas_partition
02/01/2021 00:00:09,219948000,57.720573,10.594807,0.0,355.0,160.0,Unknown,null,null,Undefined,null,null,null,1609545609,20210102
02/01/2021 00:00:20,219948000,57.720577,10.594807,0.0,348.2,160.0,Unknown,null,null,Undefined,null,null,null,1609545620,20210102
02/01/2021 00:00:29,219948000,57.720577,10.594807,0.0,6.5,160.0,Unknown,null,null,Undefined,null,null,null,1609545629,20210102
02/01/2021 00:00:39,219948000,57.720573,10.594808,0.0,20.2,160.0,Unknown,null,null,Undefined,null,null,null,1609545639,20210102
02/01/2021 00:00:50,219948000,57.72057,10.594807,0.0,346.8,160.0,Unknown,null,null,Undefined,null,null,null,1609545650,20210102
02/01/2021 00:00:59,219948000,57.720567,10.594805,0.0,340.0,159.0,Unknown,null,null,Undefined,null,null,null,1609545659,20210102
02/01/2021 00:01:10,219948000,57.720567,10.594807,0.0,1.2,159.0,Unknown,null,null,Undefined,null,null,null,1609545670,20210102
02/01/2021 00:01:20,219948000,57.720567,10.59481,0.0,359.2,159.0,Unknown,null,null,Undefined,null,null,null,1609545680,20210102
02/01/2021 00:01:29,219948000,57.720568,10.594808,0.0,329.1,159.0,Unknown,null,null,Undefined,null,null,null,1609545689,20210102
02/01/2021 00:01:39,219948000,57.720572,10.59481,0.0,329.9,159.0,Unknown,null,null,Undefined,null,null,null,1609545699,20210102


We can see two new columns: `arlas_timestamp` which is the unix timestamp corresponding to `# Timestamp`
and `arlas_partition`, corresponding to the day.

- ### Fill the missing vessel properties 

We observe that some values of the vessel characteristic fields such as `Name`, `Ship type` or `Callsign` are missing. 

The distribution of `Name` in raw data show these missing values:


In [10]:
formatted_data.groupBy("Name").count().showHTML()

Name,count
null,25
SOFIE LYKKE,9181
CASILO,4935


We apply a `StaticColumnsStandardizer` function to fill the missing vessel properties when they are available in at least one observation for a given vessel identified with its MMSI.

In [11]:
import io.arlas.data.sql._

val static_filled_data = formatted_data.process(
  new StaticColumnsStandardizer(
    "MMSI",
    Map(
      "IMO" -> ("Unknown", "Undefined"),
      "Callsign" -> ("Undefined", "Undefined"),
      "Name" -> ("Undefined", "Undefined"),
      "Ship type" -> ("Undefined", "Undefined"),
      "Cargo type" -> (null, "Undefined"),
      "Width" -> (null, "Undefined"),
      "Length" -> (null, "Undefined")
    )
  )
)
static_filled_data.sort("MMSI", "arlas_timestamp").showHTML(10)

# Timestamp,MMSI,Latitude,Longitude,SOG,COG,Heading,IMO,Callsign,Name,Ship type,Cargo type,Width,Length,arlas_timestamp,arlas_partition
02/01/2021 00:00:09,219948000,57.720573,10.594807,0.0,355.0,160.0,Unknown,OZZP,CASILO,Fishing,null,7,24,1609545609,20210102
02/01/2021 00:00:20,219948000,57.720577,10.594807,0.0,348.2,160.0,Unknown,OZZP,CASILO,Fishing,null,7,24,1609545620,20210102
02/01/2021 00:00:29,219948000,57.720577,10.594807,0.0,6.5,160.0,Unknown,OZZP,CASILO,Fishing,null,7,24,1609545629,20210102
02/01/2021 00:00:39,219948000,57.720573,10.594808,0.0,20.2,160.0,Unknown,OZZP,CASILO,Fishing,null,7,24,1609545639,20210102
02/01/2021 00:00:50,219948000,57.72057,10.594807,0.0,346.8,160.0,Unknown,OZZP,CASILO,Fishing,null,7,24,1609545650,20210102
02/01/2021 00:00:59,219948000,57.720567,10.594805,0.0,340.0,159.0,Unknown,OZZP,CASILO,Fishing,null,7,24,1609545659,20210102
02/01/2021 00:01:10,219948000,57.720567,10.594807,0.0,1.2,159.0,Unknown,OZZP,CASILO,Fishing,null,7,24,1609545670,20210102
02/01/2021 00:01:20,219948000,57.720567,10.59481,0.0,359.2,159.0,Unknown,OZZP,CASILO,Fishing,null,7,24,1609545680,20210102
02/01/2021 00:01:29,219948000,57.720568,10.594808,0.0,329.1,159.0,Unknown,OZZP,CASILO,Fishing,null,7,24,1609545689,20210102
02/01/2021 00:01:39,219948000,57.720572,10.59481,0.0,329.9,159.0,Unknown,OZZP,CASILO,Fishing,null,7,24,1609545699,20210102


If we look at the `Name` column distribution for example, we observe this harmonization:

In [12]:
static_filled_data.groupBy("Name").count().showHTML()

Name,count
SOFIE LYKKE,9182
CASILO,4959


- ### Create basic fragments and compute associated fields:

In order to identify trajectories, we first transform the AIS points into AIS fragments.

A fragment corresponds to the vessel information between two time. It has a duration, a travelled distance and many other information.

<p align="left">
  <img src="./images/fragments_creation.png" width="800" title="Fragment Creation" style="float:left">
</p>

We apply the `FlowFragmentMapper` which create the following new fields in the data:
  - **arlas_track_id**: Unique fragment identifier
  - **arlas_track_nb_geopoints**: Number of ais messages (geopoints) summarized in the fragment (2 at this step)
  - **arlas_track_trail**: Geometry of vessel travel over the fragment (a simple line at this step)
  - **arlas_track_duration_s**: Duration of the fragment (s)
  - **arlas_track_timestamp_start**: Timestamp of the first observation of the fragment
  - **arlas_track_timestamp_end**: Timestamp of the last observation of the fragment
  - **arlas_track_timestamp_center**: Middle timestamp of the fragment
  - **arlas_track_end_location_lat**: Latitude of the last observation of the fragment
  - **arlas_track_end_location_lon**: Longitude of the last observation of the fragment
  - **arlas_track_location_lat**: Latitude of the centroid of the fragment
  - **arlas_track_location_lon**: Longitude of the centroid of the fragment
  - **arlas_track_distance_gps_travelled_m**: Distance (m) travelled by the vessel over the fragment
  - **arlas_track_distance_gps_straight_line_m**: Direct distance (m) between first and last observation of the fragment
  - **arlas_track_distance_gps_straightness**: Ratio between direct distance and travelled distance over the fragment (1 at this step)
  - **arlas_track_dynamics_gps_speed**: Computed gps speed over the fragment
  - **arlas_track_dynamics_gps_bearing**: Computed gps bearing over the fragment

It also transforms numeric column by taking the average of the fragment observations (ex: `SOG` -> `arlas_track_sog`)


In [13]:
val fragment_data = static_filled_data.process(
  new FlowFragmentMapper(dataModel,
    spark,
    aggregationColumnName = dataModel.idColumn,
    averageNumericColumns = List("SOG", "COG", "Heading"))
)
fragment_data.sort("MMSI", "arlas_timestamp").showHTML(10)

# Timestamp,MMSI,Latitude,Longitude,IMO,Callsign,Name,Ship type,Cargo type,Width,Length,arlas_timestamp,arlas_partition,arlas_track_id,arlas_track_nb_geopoints,arlas_track_trail,arlas_track_duration_s,arlas_track_timestamp_start,arlas_track_timestamp_end,arlas_track_timestamp_center,arlas_track_end_location_lat,arlas_track_end_location_lon,arlas_track_location_lat,arlas_track_location_lon,arlas_track_distance_gps_travelled_m,arlas_track_distance_gps_straigth_line_m,arlas_track_distance_gps_straigthness,arlas_track_dynamics_gps_speed,arlas_track_dynamics_gps_bearing,arlas_track_SOG,arlas_track_COG,arlas_track_Heading
02/01/2021 00:00:20,219948000,57.720577,10.594807,Unknown,OZZP,CASILO,Fishing,null,7,24,1609545614,20210102,219948000#1609545...,2,LINESTRING (10.59...,11,1609545609,1609545620,1609545614,57.720577,10.594807,57.720575,10.594807,0.44549084477893386,0.44549084477893386,1.0,0.1457970037458329,0.0,0.0,351.6,160.0
02/01/2021 00:00:29,219948000,57.720577,10.594807,Unknown,OZZP,CASILO,Fishing,null,7,24,1609545624,20210102,219948000#1609545...,2,POINT (10.594807 ...,9,1609545620,1609545629,1609545624,57.720577,10.594807,57.720577,10.594807,0.0,0.0,1.0,0.0,180.0,0.0,177.35,160.0
02/01/2021 00:00:39,219948000,57.720573,10.594808,Unknown,OZZP,CASILO,Fishing,null,7,24,1609545634,20210102,219948000#1609545...,2,LINESTRING (10.59...,10,1609545629,1609545639,1609545634,57.720573,10.594808,57.720575,10.594808,0.4494589997715428,0.4494589997715428,1.0,0.16180523991775542,172.3808415094486,0.0,13.35,160.0
02/01/2021 00:00:50,219948000,57.72057,10.594807,Unknown,OZZP,CASILO,Fishing,null,7,24,1609545644,20210102,219948000#1609545...,2,LINESTRING (10.59...,11,1609545639,1609545650,1609545644,57.72057,10.594807,57.720572,10.594808,0.339390967319976,0.339390967319976,1.0,0.11107340748653759,190.11284613091573,0.0,183.5,160.0
02/01/2021 00:00:59,219948000,57.720567,10.594805,Unknown,OZZP,CASILO,Fishing,null,7,24,1609545654,20210102,219948000#1609545...,2,LINESTRING (10.59...,9,1609545650,1609545659,1609545654,57.720567,10.594805,57.720569,10.594806,0.3547395272308086,0.3547395272308086,1.0,0.14189581089232342,199.63217913501893,0.0,343.4,159.5
02/01/2021 00:01:10,219948000,57.720567,10.594807,Unknown,OZZP,CASILO,Fishing,null,7,24,1609545664,20210102,219948000#1609545...,2,LINESTRING (10.59...,11,1609545659,1609545670,1609545664,57.720567,10.594807,57.720567,10.594806,0.11918560155483904,0.11918560155483904,1.0,0.03900619687249277,89.9999991545464,0.0,170.6,159.0
02/01/2021 00:01:20,219948000,57.720567,10.59481,Unknown,OZZP,CASILO,Fishing,null,7,24,1609545675,20210102,219948000#1609545...,2,LINESTRING (10.59...,10,1609545670,1609545680,1609545675,57.720567,10.59481,57.720567,10.594809,0.17877840238518763,0.17877840238518763,1.0,0.06436022485866755,89.99999873181963,0.0,180.2,159.0
02/01/2021 00:01:29,219948000,57.720568,10.594808,Unknown,OZZP,CASILO,Fishing,null,7,24,1609545684,20210102,219948000#1609545...,2,LINESTRING (10.59...,9,1609545680,1609545689,1609545684,57.720568,10.594808,57.720568,10.594809,0.1631229224786664,0.1631229224786664,1.0,0.06524916899146656,313.05917134989454,0.0,344.15,159.0
02/01/2021 00:01:39,219948000,57.720572,10.59481,Unknown,OZZP,CASILO,Fishing,null,7,24,1609545694,20210102,219948000#1609545...,2,LINESTRING (10.59...,10,1609545689,1609545699,1609545694,57.720572,10.59481,57.72057,10.594809,0.4611586467396393,0.4611586467396393,1.0,0.16601711282627016,14.977998946642629,0.0,329.5,159.0
02/01/2021 00:01:50,219948000,57.720573,10.594813,Unknown,OZZP,CASILO,Fishing,null,7,24,1609545704,20210102,219948000#1609545...,2,LINESTRING (10.59...,11,1609545699,1609545710,1609545704,57.720573,10.594813,57.720573,10.594812,0.21063140327181465,0.21063140327181465,1.0,0.06893391379804843,58.07852797665032,0.0,344.45,159.0


- ### Add the motion visibility for each fragment

We add a notion of visibility. When the duration between two observations is higher than 30min, we consideer that  the move is "invisible". We don't have enough information to describe the real behaviour of the vessel over this fragment. We say that `visibility proportion=0`. Otherwise, a visible fragment has a `visibility proportion=1`.

We also want to identify the transitions between these states and we add the `visibility change`: 

- "appear": invisible->visible

- "disappear": visible->invisible

In [14]:
val visibility_data = fragment_data.process(
  // Identify fragment as invisible when duration since previous observation is higher than 30min
  new WithVisibilityProportion(durationThreshold = 1800),
  // Detect visibility change ("appear": invisible->visible, "disappear": visible->invisible)
  new WithVisibilityChange(dataModel = dataModel)
)
visibility_data
        .select("MMSI", "arlas_timestamp", "arlas_track_duration_s", "arlas_track_visibility_proportion", "arlas_track_visibility_change")
        .sort("MMSI", "arlas_timestamp")
        .showHTML(10)

MMSI,arlas_timestamp,arlas_track_duration_s,arlas_track_visibility_proportion,arlas_track_visibility_change
219948000,1609545614,11,1.0,null
219948000,1609545624,9,1.0,null
219948000,1609545634,10,1.0,null
219948000,1609545644,11,1.0,null
219948000,1609545654,9,1.0,null
219948000,1609545664,11,1.0,null
219948000,1609545675,10,1.0,null
219948000,1609545684,9,1.0,null
219948000,1609545694,10,1.0,null
219948000,1609545704,11,1.0,null


We can display for each vessel the evolution of fragment duration (time between AIS messages) over time. Here one of the vessel is very regular (around 10 seconds) while the other has small gaps in its ais message emission :

In [15]:
val (x1, y1) = visibility_data.filter(col("MMSI") === 219948000).sort("arlas_timestamp").select("arlas_timestamp","arlas_track_duration_s").collect.map(r=>(r(0).toString.toInt, r(1).toString.toFloat)).toList.unzip
val (x2, y2) = visibility_data.filter(col("MMSI") === 220343000).sort("arlas_timestamp").select("arlas_timestamp","arlas_track_duration_s").collect.map(r=>(r(0).toString.toInt, r(1).toString.toFloat)).toList.unzip
Seq(Scatter(x1, y1,name="MMSI: 219948000", mode = ScatterMode(ScatterMode.Markers,ScatterMode.Lines)),
    Scatter(x2, y2,name="MMSI: 220343000", mode = ScatterMode(ScatterMode.Markers,ScatterMode.Lines))
   ).plot(div="Duration",
          layout=Layout(title="Fragment duration over time",
                        xaxis=Axis("Unix Timestamp"),
                        yaxis=Axis("Fragment duration (s)")))

It results in the following visibility proportions:

In [16]:
val (x1, y1) = visibility_data.filter(col("MMSI") === 219948000).sort("arlas_timestamp").select("arlas_timestamp","arlas_track_visibility_proportion").collect.map(r=>(r(0).toString.toInt, r(1).toString.toFloat)).toList.unzip
val (x2, y2) = visibility_data.filter(col("MMSI") === 220343000).sort("arlas_timestamp").select("arlas_timestamp","arlas_track_visibility_proportion").collect.map(r=>(r(0).toString.toInt, r(1).toString.toFloat)).toList.unzip
Seq(Scatter(x1, y1,name="MMSI: 219948000", mode = ScatterMode(ScatterMode.Markers, ScatterMode.Lines)),
    Scatter(x2, y2,name="MMSI: 220343000", mode = ScatterMode(ScatterMode.Markers, ScatterMode.Lines))
   ).plot(div="Visibility",
          layout=Layout(title="Visibility over time",
                        xaxis=Axis("Unix Timestamp"),
                        yaxis=Axis("Visibility proportion")))

- ### Identify the moving state of the vessel for each fragment

The first step to analyse vessel trajectories is to detect whether the boat is still or moving.

We use a Hidden Markov Model based on vessel speed to identify the vessel mobility. Here the model is very simple:

<p align="left">
  <img src="./images/hmm_model.png" width="750" title="HMM Model" style="float:left">
</p>

HMM model is robust to noise in vessel speed to avoid switching between “still” and “move” state because of wrong measurement.

In [17]:
val moving_data = visibility_data.process(
  // Detect if the boat is still or moving with an hmm model based on speed
  new WithMovingState(spark,
    idColumn = "MMSI",
    speedColumn = "arlas_track_SOG",
    targetMovingState = "arlas_moving_state",
    stillMoveModelPath = "/home/jovyan/work/model/hmm_still_move.json"),
  // Create a common identifier for consecutive fragment sharing the same moving state
  new WithStateIdOnStateChangeOrUnique(idColumn = "MMSI",
    stateColumn = "arlas_moving_state",
    orderColumn = "arlas_track_timestamp_start",
    targetIdColumn = "arlas_motion_id"),
  // Update the motion identifier as MMSI#timestampStart_timestampEnd
  new IdUpdater(idColumn = "arlas_motion_id", dataModel = dataModel),
  // Compute the duration of each motion
  new WithDurationFromId(idColumn = "arlas_motion_id", targetDurationColumn = "arlas_motion_duration")
)
moving_data
        .select("MMSI", "arlas_timestamp", "arlas_track_SOG", "arlas_moving_state", "arlas_motion_id", "arlas_motion_duration")
        .sort("MMSI", "arlas_timestamp")
        .showHTML(10)

MMSI,arlas_timestamp,arlas_track_SOG,arlas_moving_state,arlas_motion_id,arlas_motion_duration
219948000,1609545614,0.0,STILL,219948000#1609545...,35654
219948000,1609545624,0.0,STILL,219948000#1609545...,35654
219948000,1609545634,0.0,STILL,219948000#1609545...,35654
219948000,1609545644,0.0,STILL,219948000#1609545...,35654
219948000,1609545654,0.0,STILL,219948000#1609545...,35654
219948000,1609545664,0.0,STILL,219948000#1609545...,35654
219948000,1609545675,0.0,STILL,219948000#1609545...,35654
219948000,1609545684,0.0,STILL,219948000#1609545...,35654
219948000,1609545694,0.0,STILL,219948000#1609545...,35654
219948000,1609545704,0.0,STILL,219948000#1609545...,35654


For a given vessel, we can plot the evolution of speed colored by the predicted motion state:

In [18]:
val (x1, y1) = moving_data.filter(col("MMSI") === 220343000).filter(col("arlas_moving_state") === "STILL").sort("arlas_timestamp").select("arlas_timestamp","arlas_track_SOG").collect.map(r=>(r(0).toString.toInt, r(1).toString.toFloat)).toList.unzip
val (x2, y2) = moving_data.filter(col("MMSI") === 220343000).filter(col("arlas_moving_state") === "MOVE").sort("arlas_timestamp").select("arlas_timestamp","arlas_track_SOG").collect.map(r=>(r(0).toString.toInt, r(1).toString.toFloat)).toList.unzip
Seq(Scatter(x1, y1,name="STILL", mode = ScatterMode(ScatterMode.Markers), marker=Marker(
     color = Color.RGBA(255, 0, 0, 0.80),
     symbol = Symbol.Circle(),
     size = 6
    )),
    Scatter(x2, y2,name="MOVE", mode = ScatterMode(ScatterMode.Markers), marker=Marker(
     color = Color.RGBA(0, 204, 0, 0.80),
     symbol = Symbol.Circle(),
     size = 6
    ))
   ).plot(div="Still Move",
          layout=Layout(title="SOG over time for MMSI=220343000",
                        xaxis=Axis("Unix Timestamp"),
                        yaxis=Axis("Speed Over Ground (knot)")))

- ### Build course information from moving state

We identify fragments as a STOP if the vessel is still for more than 10 minutes, and COURSE else. The result is stored in a column `arlas_course_or_stop ∈ {COURSE , STOP}`

We identify fragments as PAUSE when the vessel is still for less than 10 minutes, and MOTION when vessel is moving. The result is stored in a column `arlas_course_state  ∈ {MOTION , PAUSE}`.

We create the course identifier (stored in `arlas_course_id`).

We compute the course duration (stored in `arlas_course_duration`).


In [19]:
val course_data = moving_data.process(
  new WithCourseOrStop(courseTimeoutS = 600),
  new WithCourseState(),
  new WithStateIdOnStateChangeOrUnique(idColumn = "MMSI",
    stateColumn = "arlas_course_or_stop",
    orderColumn = "arlas_track_timestamp_start",
    targetIdColumn = "arlas_course_id"),
  // Update the course identifier as MMSI#timestampStart_timestampEnd
  new IdUpdater(dataModel = dataModel, idColumn = "arlas_course_id"),
  new WithDurationFromId(idColumn = "arlas_course_id", targetDurationColumn = "arlas_course_duration")
)
course_data
        .select(
          "MMSI", "arlas_timestamp", "arlas_moving_state", "arlas_motion_id", "arlas_motion_duration",
          "arlas_course_or_stop", "arlas_course_state", "arlas_course_id", "arlas_course_duration"
        )
        .sort("MMSI", "arlas_timestamp")
        .showHTML(10)

MMSI,arlas_timestamp,arlas_moving_state,arlas_motion_id,arlas_motion_duration,arlas_course_or_stop,arlas_course_state,arlas_course_id,arlas_course_duration
219948000,1609545614,STILL,219948000#1609545...,35654,STOP,null,219948000#1609545...,35654
219948000,1609545624,STILL,219948000#1609545...,35654,STOP,null,219948000#1609545...,35654
219948000,1609545634,STILL,219948000#1609545...,35654,STOP,null,219948000#1609545...,35654
219948000,1609545644,STILL,219948000#1609545...,35654,STOP,null,219948000#1609545...,35654
219948000,1609545654,STILL,219948000#1609545...,35654,STOP,null,219948000#1609545...,35654
219948000,1609545664,STILL,219948000#1609545...,35654,STOP,null,219948000#1609545...,35654
219948000,1609545675,STILL,219948000#1609545...,35654,STOP,null,219948000#1609545...,35654
219948000,1609545684,STILL,219948000#1609545...,35654,STOP,null,219948000#1609545...,35654
219948000,1609545694,STILL,219948000#1609545...,35654,STOP,null,219948000#1609545...,35654
219948000,1609545704,STILL,219948000#1609545...,35654,STOP,null,219948000#1609545...,35654


- ### Concatenate the Pauses and Stops into single fragments

In order to compress the information, we can we can group all consecutive fragments corresponding to a stop or a pause together in a single longer fragment.

It results in much less redondancy in the data (many rows are agregated in only one fragment).

In [20]:
val zipped_stop = course_data.process(
  new StopPauseSummaryTransformer(
    spark,
    dataModel,
    weightAveragedColumns = Some(
      Seq(
        "arlas_track_dynamics_gps_speed",
        "arlas_track_dynamics_gps_bearing",
        "arlas_track_SOG",
        "arlas_track_COG",
        "arlas_track_Heading",
        "arlas_track_visibility_proportion"
      )),
    propagatedColumns = Some(Seq("IMO", "Callsign", "Name", "Ship type", "Cargo type", "Width", "Length"))
  )
)
zipped_stop.select("MMSI", "arlas_timestamp", "arlas_track_duration_s", "arlas_moving_state", "arlas_motion_id",
                   "arlas_motion_duration", "arlas_course_or_stop", "arlas_course_state", "arlas_course_id", 
                   "arlas_course_duration", "arlas_track_trail")
           .sort("MMSI", "arlas_timestamp")
           .showHTML(10)

MMSI,arlas_timestamp,arlas_track_duration_s,arlas_moving_state,arlas_motion_id,arlas_motion_duration,arlas_course_or_stop,arlas_course_state,arlas_course_id,arlas_course_duration,arlas_track_trail
219948000,1609563436,35654,STILL,219948000#1609545...,35654,STOP,null,219948000#1609545...,35654,LINESTRING (10.59...
219948000,1609581264,3,MOVE,219948000#1609581...,46,COURSE,MOTION,219948000#1609581...,326,LINESTRING (10.59...
219948000,1609581268,4,MOVE,219948000#1609581...,46,COURSE,MOTION,219948000#1609581...,326,LINESTRING (10.59...
219948000,1609581271,3,MOVE,219948000#1609581...,46,COURSE,MOTION,219948000#1609581...,326,LINESTRING (10.59...
219948000,1609581275,4,MOVE,219948000#1609581...,46,COURSE,MOTION,219948000#1609581...,326,LINESTRING (10.59...
219948000,1609581278,2,MOVE,219948000#1609581...,46,COURSE,MOTION,219948000#1609581...,326,LINESTRING (10.59...
219948000,1609581281,4,MOVE,219948000#1609581...,46,COURSE,MOTION,219948000#1609581...,326,LINESTRING (10.59...
219948000,1609581284,3,MOVE,219948000#1609581...,46,COURSE,MOTION,219948000#1609581...,326,LINESTRING (10.59...
219948000,1609581287,3,MOVE,219948000#1609581...,46,COURSE,MOTION,219948000#1609581...,326,LINESTRING (10.59...
219948000,1609581291,4,MOVE,219948000#1609581...,46,COURSE,MOTION,219948000#1609581...,326,LINESTRING (10.59...


- ### Concatenate all fragments associated to a course into a single course fragment

We can apply a same kind of process to group all fragments associated together into a single longer course fragment.

<p align="left">
  <img src="./images/fragment_concatenation.png" width="800" title="Fragment Creation" style="float:left">
</p>

The stops are stored as "stop before" and "stop after" a course.

It results in course fragment, with an origin, destination, duration, distance, track geometry.

In [21]:
val course_extracted = zipped_stop.process(
  new CourseExtractorTransformer(
    spark,
    dataModel,
    weightAveragedColumns = Some(
      Seq(
        "arlas_track_dynamics_gps_speed",
        "arlas_track_dynamics_gps_bearing",
        "arlas_track_SOG",
        "arlas_track_COG",
        "arlas_track_Heading",
        "arlas_track_visibility_proportion"
      )),
    propagatedColumns = Some(Seq("IMO", "Callsign", "Name", "Ship type", "Cargo type", "Width", "Length"))
  )
)
course_extracted
  .select(
    "MMSI", "arlas_timestamp", "arlas_track_duration_s", "arlas_track_nb_geopoints", "arlas_departure_stop_before_duration_s",
    "arlas_arrival_stop_after_duration_s", "arlas_moving_state", "arlas_course_or_stop", "arlas_course_state",
    "arlas_course_id", "arlas_course_duration", "arlas_track_trail"
  )
  .sort("MMSI", "arlas_timestamp")
  .showHTML()

MMSI,arlas_timestamp,arlas_track_duration_s,arlas_track_nb_geopoints,arlas_departure_stop_before_duration_s,arlas_arrival_stop_after_duration_s,arlas_moving_state,arlas_course_or_stop,arlas_course_state,arlas_course_id,arlas_course_duration,arlas_track_trail
219948000,1609581426,326,88,35654,2020,MOVE,COURSE,MOTION,219948000#1609581...,326,LINESTRING (10.59...
219948000,1609592124,17031,1021,2020,null,MOVE,COURSE,MOTION,219948000#1609583...,17031,LINESTRING (10.59...
220343000,1609585328,43853,4849,17801,24736,MOVE,COURSE,MOTION,220343000#1609563...,43853,LINESTRING (10.58...


We observe that all the raw AIS points are now aggregated in several courses that can easily be explored in tools such as QGIS or ARLAS Exploration.

As a reminder, we had only raw geopoints at the origin:

<p align="left">
  <img src="./images/qgis_raw.png" width="900" title="Courses" style="float:left">
</p>



After process we can see the 3 resulting courses (colored by `arlas_course_id`) on Qgis. The stops have been simplified and we directly see the separated travels, their origin (green circle) and destination (red circle). Other courses information such as duration and distance are also available in data.

<p align="left">
  <img src="./images/qgis_course.png" width="900" title="Courses" style="float:left">
</p>

Much more AIS courses are available in ARLAS Exploration on [ARLAS demo](https://demo.cloud.arlas.io/arlas/wui/?config_id=6uKsOIIOQkKL5MvQin8R&extend=1.1618731760410697,53.33855185171089,18.827121206255924,58.261748527259016).

<p align="left">
  <img src="./images/course_fluxes.png" width="900" title="Courses fluxes" style="float:left">
</p>

- ### Write course result in a csv file

We can store these processed courses in a csv file:

In [22]:
course_extracted.writeToCsv("/home/jovyan/work/data/ais_course_data")